In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import json
import numpy as np
from scripts.load_data import *

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor

In [3]:
configuration = load_configuration("../configuration/configuration.json")

paths = configuration["paths"]
data_student_numbers_f = pd.read_excel(paths["path_student_count_first-years"])
data_student_numbers_h = pd.read_excel(paths["path_student_count_higher-years"])
data_student_numbers_v = pd.read_excel(paths["path_student_volume"])

In [4]:
combined_data_student_numbers = data_student_numbers_f.merge(data_student_numbers_h, on=["Croho groepeernaam", "Herkomst", "Collegejaar"])
combined_data_student_numbers = combined_data_student_numbers.rename(columns={"Aantal_studenten_x": "Aantal_studenten_f", "Aantal_studenten_y": "Aantal_studenten_h"})
combined_data_student_numbers

,Collegejaar,Croho groepeernaam,Herkomst,Aantal_studenten_f,Aantal_studenten_h
0,2012,M Geneeskunde,NL,180,868
1,2013,M Geneeskunde,NL,173,892
2,2014,M Geneeskunde,NL,175,916
3,2015,M Geneeskunde,NL,21,1086
4,2016,M Geneeskunde,NL,148,984
...,...,...,...,...,...
2156,2016,M Leraar Voorbereidend Hoger Onderwijs in Gods...,NL,1,1
2157,2014,M Leraar Voorbereidend Hoger Onderwijs in Natu...,NL,3,1
2158,2015,M Leraar Voorbereidend Hoger Onderwijs in Natu...,NL,1,1
2159,2017,M Leraar Voorbereidend Hoger Onderwijs in Natu...,NL,2,1


In [5]:
combined_data_student_numbers[combined_data_student_numbers["Croho groepeernaam"] == "B Sociologie"]

,Collegejaar,Croho groepeernaam,Herkomst,Aantal_studenten_f,Aantal_studenten_h
1819,2012,B Sociologie,NL,49,76
1820,2013,B Sociologie,NL,51,88
1821,2014,B Sociologie,NL,38,108
1822,2015,B Sociologie,NL,32,105
1823,2016,B Sociologie,NL,44,95
1824,2017,B Sociologie,NL,31,98
1825,2018,B Sociologie,NL,43,89
1826,2019,B Sociologie,NL,48,99
1827,2020,B Sociologie,NL,63,99
1828,2021,B Sociologie,NL,86,111


In [6]:
predict_year = 2023
data = combined_data_student_numbers

In [7]:
years = data["Collegejaar"].unique()
years = np.sort(years)
first_training_year = years[0]

nr_of_years = predict_year - first_training_year - 1

In [8]:
nr_of_years

10

In [9]:
train = data[(data["Collegejaar"] >= first_training_year) & (data["Collegejaar"] <= predict_year - 1)]
train = train.pivot(index=['Croho groepeernaam', 'Herkomst'], columns='Collegejaar', values='Aantal_studenten_h').reset_index()
train.columns = map(str, train.columns)
train = train.fillna(0)
dict = {}
for i in range(nr_of_years):
    dict[str(first_training_year + i)] = str(i)
train = train.rename(columns=dict)

In [10]:
X_train = train.drop([str(predict_year - 1)], axis=1)
y_train = train.pop(str(predict_year - 1))

In [11]:
test = data[(data["Collegejaar"] >= first_training_year + 1) & (data["Collegejaar"] <= predict_year)]
test = test.pivot(index=['Croho groepeernaam', 'Herkomst'], columns='Collegejaar', values='Aantal_studenten_h').reset_index()
test.columns = map(str, test.columns)
test = test.fillna(0)
dict = {}
for i in range(nr_of_years):
    dict[str(first_training_year + 1 + i)] = str(i)
test = test.rename(columns=dict)

In [12]:
X_test = test.drop([str(predict_year)], axis=1)
y_test = test.pop(str(predict_year))

In [13]:
X_train

,Croho groepeernaam,Herkomst,0,1,2,3,4,5,6,7,8,9
0,B Algemene Cultuurwetenschappen,EER,1.0,2.0,2.0,3.0,4.0,9.0,21.0,31.0,29.0,21.0
1,B Algemene Cultuurwetenschappen,NL,117.0,113.0,104.0,87.0,72.0,66.0,93.0,79.0,81.0,83.0
2,B Algemene Cultuurwetenschappen,Niet-EER,0.0,0.0,0.0,0.0,3.0,12.0,9.0,13.0,12.0,7.0
3,B Artificial Intelligence,EER,5.0,7.0,10.0,13.0,17.0,37.0,51.0,62.0,57.0,62.0
4,B Artificial Intelligence,NL,89.0,99.0,124.0,145.0,179.0,220.0,250.0,268.0,295.0,334.0
...,...,...,...,...,...,...,...,...,...,...,...,...
250,M Theologie,NL,3.0,0.0,8.0,12.0,0.0,5.0,5.0,5.0,6.0,3.0
251,M Theologie,Niet-EER,0.0,0.0,2.0,8.0,10.0,8.0,6.0,2.0,0.0,1.0
252,M Theologie & Religiewetenschappen,EER,0.0,4.0,4.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0
253,M Theologie & Religiewetenschappen,NL,1.0,45.0,53.0,63.0,69.0,71.0,79.0,88.0,79.0,132.0


In [14]:
numeric_cols = [str(x) for x in range(nr_of_years)]
categorical_cols = ['Croho groepeernaam', 'Herkomst']

numeric_transformer = "passthrough"  # No transformation for numeric columns
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_cols),
        ('categorical', categorical_transformer, categorical_cols)
    ])

X_train = preprocessor.fit_transform(X_train)
preprocessed_test = preprocessor.transform(X_test)

In [15]:
print(preprocessed_test.shape)
print(len(test))

(256, 130)
256


In [16]:
model = XGBRegressor(learning_rate=0.25)

model.fit(X_train, y_train)

predictie = model.predict(preprocessed_test)

print(predictie)
print(len(predictie))

[ 2.79297447e+01  7.56931839e+01  6.14622498e+00  8.06436768e+01
  3.03429413e+02  1.52669497e+01  7.32112122e+01  1.05597070e+03
  2.87876358e+01  4.49794717e-02  1.86210831e+02 -3.18564117e-01
  1.12793297e+02  4.24052216e+02  2.36443939e+01 -6.25913918e-01
  1.60337463e+02  2.50650234e+01  6.62978287e+01  1.01001978e+01
  4.54240837e+01  4.23701569e+02  1.09450760e+01  1.99238971e-01
  2.02662155e+02  6.01373749e+01  3.00625793e+02  2.09969997e+01
 -2.39877060e-01  8.58375931e+01 -1.92745805e-01  3.19842415e+01
  3.85111580e+01  4.34245453e+02  1.13571768e+01  1.68625698e+01
  1.47671783e+02  1.52237105e+00 -4.55687158e-02  1.43315155e+02
  2.31405888e-02  4.86579170e+01  2.14832382e+02  8.86017799e+00
  1.02580442e+01  8.80732300e+02 -6.90180138e-02  8.01869392e+00
  3.04689880e+02 -6.90180138e-02  1.03765583e+01  2.89998413e+02
 -5.07822037e-01  5.26478386e+01 -1.20496303e-02 -1.20496303e-02
  3.73306535e-02  5.95695267e+01  5.19140778e+01  1.53939926e+02
  8.18065548e+00  5.00490

In [17]:
total_mae = 0.0
total_mape = 0.0
i = 0
for _, row in test.iterrows():
    programme = row["Croho groepeernaam"]
    origin = row["Herkomst"]
    higher_year_row = data_student_numbers_h[(data_student_numbers_h["Croho groepeernaam"] == programme) &
                                                    (data_student_numbers_h["Herkomst"] == origin) &
                                                    (data_student_numbers_h["Collegejaar"] == predict_year)]

    try:
        mae = abs(predictie[i] - higher_year_row["Aantal_studenten"].iloc[0])
        mape = abs((predictie[i] - higher_year_row["Aantal_studenten"].iloc[0]) / higher_year_row["Aantal_studenten"].iloc[0])
        
        print(f"{programme}, {origin}: {mae}")

        total_mae += mae
        total_mape += mape
        i += 1
    except:
        print(f"{programme}, {origin}: Error")

print(f"Final MAE: {total_mae / i}")
print(f"Final MAPE: {total_mape / i}")

B Algemene Cultuurwetenschappen, EER: 8.070255279541016
B Algemene Cultuurwetenschappen, NL: 1.3068161010742188
B Algemene Cultuurwetenschappen, Niet-EER: 2.8537750244140625
B Artificial Intelligence, EER: 3.3563232421875
B Artificial Intelligence, NL: 28.429412841796875
B Artificial Intelligence, Niet-EER: 0.7330503463745117
B Bedrijfskunde, EER: 6.788787841796875
B Bedrijfskunde, NL: 40.970703125
B Bedrijfskunde, Niet-EER: 10.787635803222656
B Bestuurskunde, EER: Error
B Bestuurskunde, NL: 193.9550205282867
B Bestuurskunde, Niet-EER: 185.21083068847656
B Biology, EER: 124.3185641169548
B Biology, NL: 128.20670318603516
B Biology, Niet-EER: 401.0522155761719
B Biomedische Wetenschappen, EER: 22.644393920898438
B Biomedische Wetenschappen, NL: 194.62591391801834
B Chemistry, EER: 135.33746337890625
B Chemistry, NL: 51.93497657775879
B Chemistry, Niet-EER: 56.297828674316406
B Communicatie- en Informatiewetenschappen, EER: 35.89980220794678
B Communicatie- en Informatiewetenschappen, NL